<h1>Imports</h1>

In [9]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import os
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

<h1> Dataloader </h1>

In [21]:
import os
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataset_folder, transform=None):
        self.dataset_folder = dataset_folder
        self.transform = transform
        self.data = []
        self.label_encoder = LabelEncoder()
        
        for img_name in os.listdir(dataset_folder):
            if img_name.endswith('.jpg'): 
                label = img_name.split('_')[0]
                full_img_path = os.path.join(dataset_folder, img_name)
                self.data.append((full_img_path, label))
        
        self.labels = [label for _, label in self.data]
        self.label_encoder.fit(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        label_encoded = self.label_encoder.transform([label])[0]
        if self.transform:
            image = self.transform(image)
        return image, label_encoded

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Grayscale(num_output_channels=1),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset_folder = r'D:\405 FOUND\Comp-Vision\OCR-With-CNN\CNN-TextRecognition\Dataset'

dataset = CustomDataset(dataset_folder, transform=transform)

train_data, val_data = train_test_split(dataset.data, test_size=0.2, random_state=42)
train_dataset = CustomDataset(dataset_folder, transform=transform)
train_dataset.data = train_data
val_dataset = CustomDataset(dataset_folder, transform=transform)
val_dataset.data = val_data

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Train dataset size: {len(train_loader.dataset)}")
print(f"Validation dataset size: {len(val_loader.dataset)}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'CNN-TextRecognition\\Dataset'

<h1> Architecture </h1>
CNN-Fully Connected Layers

In [16]:
class CNNForOCR(nn.Module):
    def __init__(self, num_classes, dropout=0.5):
        super(CNNForOCR, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 256, kernel_size=1), 
            nn.ReLU(),
        )
        self.fc1 = nn.Linear(256 * 7 * 7, 1024)  
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
num_classes = len(dataset.label_encoder.classes_)
model = CNNForOCR(num_classes=num_classes, dropout=0.5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 25


<h1> Training </h1>

In [17]:
patience = 5
best_val_loss = float('inf')
epochs_without_improvement = 0
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    avg_val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
    if epochs_without_improvement >= patience:
        print("Early stopping triggered. No improvement in validation loss for", patience, "epochs.")
        break

print(f"Final Train Accuracy: {train_accuracies[-1]:.2f}%")
print(f"Final Validation Accuracy: {val_accuracies[-1]:.2f}%")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\405 FOUND\\Comp-Vision\\OCR-With-CNN\\CNN-TextRecognition\\CNN-TextRecognition\\Dataset\\CLE_Normalized_Size_Variant_Ligatures\\3318_Original_RetuSinu.jpg'